# Импорты

In [1]:
import os
import torch
import numpy as np
import torchvision
from tqdm import tqdm
import torch.nn as nn
from matplotlib import gridspec
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import Country211
from sklearn.metrics import classification_report

# ResNet

In [2]:
class BasicBlock(nn.Module):
    """Basic Block class"""

    def __init__(self, in_channels, out_channels, use_batch_norm=False, stride=1):
        super(BasicBlock, self).__init__()
        self.use_batch_norm = use_batch_norm
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        if use_batch_norm:
            self.bn1 = nn.BatchNorm2d(out_channels)
            self.bn2 = nn.BatchNorm2d(out_channels)


        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            if use_batch_norm:
                self.shortcut = nn.Sequential(
                    nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                    nn.BatchNorm2d(out_channels)
                )
            else:
                self.shortcut = nn.Sequential(
                    nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
                )


    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out) if self.use_batch_norm else out
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out) if self.use_batch_norm else out

        out += self.shortcut(x)
        out = self.relu(out)

        return out


class ResNet18(nn.Module):
    def __init__(self, block, num_blocks=2, num_classes=1000, use_batch_norm=False):
        super(ResNet18, self).__init__()
        self.use_batch_norm = use_batch_norm
        self.in_channels = 64

        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        if self.use_batch_norm:
          self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # далее идут 4 слоя с двумя остаточными блоками
        self.layer1 = self.make_layer(block, 64, num_blocks, use_batch_norm, stride=1)
        self.layer2 = self.make_layer(block, 128, num_blocks, use_batch_norm, stride=2)
        self.layer3 = self.make_layer(block, 256, num_blocks, use_batch_norm, stride=2)
        self.layer4 = self.make_layer(block, 512, num_blocks, use_batch_norm, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

        # Инициализация Xavier
        self.initialize_weights()

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)  # инициализация смещения нулями

    def make_layer(self, block, out_channels, num_blocks, use_batch_norm, stride):
        """Реализуем слоя с n остаточными блоками"""
        layers = []
        # кладем в каждый слой блок
        layers.append(block(self.in_channels, out_channels, use_batch_norm, stride))
        self.in_channels = out_channels
        for i in range(1, num_blocks):
            layers.append(block(out_channels, out_channels, use_batch_norm, stride=1))
        return nn.Sequential(*layers)

    def forward(self, x):
        """Прямой проход по сети"""
        out = self.conv1(x)
        out = self.bn1(out) if self.use_batch_norm else out
        out = self.relu(out)
        out = self.maxpool(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)

        return out


# Всякие полезные функции

### Отрисовка результатов

In [3]:
def show_dependencies(num_layers, train_results, test_results):
    plt.figure(figsize=(10, 6))
    plt.plot(num_layers, test_results, marker='o', label='Test Results', color='b')
    plt.plot(num_layers, train_results, marker='o', label='Train Results', color='g')
    plt.title('Зависимость тестовых и тренировочных результатов от количества слоев')
    plt.xlabel('Количество слоев (num_layers)')
    plt.ylabel('Результаты')
    plt.xticks(num_layers)
    plt.grid(True)
    plt.legend()

### Батчевое обучение

In [4]:
def train_on_batch(model, x_batch, y_batch, optimizer, loss_function):
    model.train()
    model.zero_grad()

    output = model(x_batch.to(device))

    loss = loss_function(output, y_batch.to(device))
    loss.backward()

    optimizer.step()
    return loss.cpu().item()

def train_epoch(train_generator, model, loss_function, optimizer, callback = None):
    epoch_loss = 0
    total = 0
    for it, (batch_of_x, batch_of_y) in enumerate(train_generator):
        batch_loss = train_on_batch(model, batch_of_x.to(device), batch_of_y.to(device), optimizer, loss_function)

        if callback is not None:
            callback(model, batch_loss)

        epoch_loss += batch_loss*len(batch_of_x)
        total += len(batch_of_x)

    return epoch_loss/total


def trainer(count_of_epoch,
            batch_size,
            dataset,
            model,
            loss_function,
            optimizer,
            lr = 0.001,
            callback = None):

    optima = optimizer(model.parameters(), lr=lr)
    for it in range(count_of_epoch):
        batch_generator = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
        epoch_loss = train_epoch(train_generator=batch_generator,
                    model=model,
                    loss_function=loss_function,
                    optimizer=optima,
                    callback=callback)

### Тестирование

In [5]:
def tester(model, test_dataset, loss_function):
    batch_generator = torch.utils.data.DataLoader(dataset = test_dataset,
                                                  batch_size=64)
    pred = []
    real = []
    test_loss = 0
    for it, (x_batch, y_batch) in enumerate(batch_generator):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        output = model(x_batch)
        test_loss += loss_function(output, y_batch).cpu().item()*len(x_batch)
        pred.extend(torch.argmax(output, dim=-1).cpu().numpy().tolist())
        real.extend(y_batch.cpu().numpy().tolist())

    report = classification_report(real, pred, output_dict=True)
    return report['macro avg']['f1-score'], classification_report(real, pred)

# Выбираем набор данных

In [6]:
# Load the dataset
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True,
                                                  transform=transforms.Compose([transforms.ToTensor(), ]))

test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True,
                                                 transform=transforms.Compose([transforms.ToTensor(), ]))

100%|█████████████████████████| 26421880/26421880 [00:01<00:00, 13383079.10it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|█████████████████████████████████| 29515/29515 [00:00<00:00, 465434.54it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|███████████████████████████| 4422102/4422102 [00:00<00:00, 10758516.90it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████████████████████████████| 5148/5148 [00:00<00:00, 5505425.04it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



# Выбираем устройство для исполнения

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# ResNet-18

## Обучение модели ResNet и тест

In [ ]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam
model_resnet = ResNet18(block=BasicBlock,num_blocks=4, num_classes=10, use_batch_norm=True).to(device)

trainer(count_of_epoch = 1,
        batch_size = 64,
        dataset = train_dataset,
        model = model_resnet,
        loss_function = loss_function,
        optimizer = optimizer,
        lr = 0.001,
        callback = None)

macro_avg, report = tester(model_resnet, test_dataset, loss_function)
print(report)

              precision    recall  f1-score   support

           0       0.87      0.68      0.76      1000
           1       0.98      0.97      0.98      1000
           2       0.78      0.75      0.77      1000
           3       0.89      0.82      0.85      1000
           4       0.69      0.78      0.73      1000
           5       0.96      0.96      0.96      1000
           6       0.57      0.71      0.63      1000
           7       0.94      0.95      0.94      1000
           8       0.96      0.96      0.96      1000
           9       0.97      0.95      0.96      1000

    accuracy                           0.85     10000
   macro avg       0.86      0.85      0.85     10000
weighted avg       0.86      0.85      0.85     10000



## Смотрим на зависимость ResNet18 от разного количество слоев

In [11]:
num_layers = range(2, 10, 2)
results_train = []
results_test = []
for num in tqdm(num_layers, desc="Перебираем разное количество слоев"):
    loss_function = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam
    model_resnet = ResNet18(block=BasicBlock, num_blocks=num, num_classes=10, use_batch_norm=True).to(device)
    trainer(count_of_epoch = 1,
            batch_size = 64,
            dataset = train_dataset,
            model = model_resnet,
            loss_function = loss_function,
            optimizer = optimizer,
            lr = 0.001,
            callback = None)

    macro_avg_train, _ = tester(model_resnet, train_dataset, loss_function)
    results_train.append(macro_avg_train)

    macro_avg_test, _ = tester(model_resnet, test_dataset, loss_function)
    results_test.append(macro_avg_test)

Перебираем разное количество слоев:   0%|                 | 0/4 [04:38<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
show_dependencies(num_layers, results_train, results_test)